In [5]:
import numpy as np 
import os
import scipy.stats as st
from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter  
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc
import cmprsk.cmprsk as cmprsk
from scipy import integrate
import matplotlib.pyplot as plt
import time
device="cuda" if torch.cuda.is_available else "cpu"
torch.set_default_tensor_type(torch.DoubleTensor)
def g1(x):
    g = x[:,0] + 2*x[:,1] + 3*x[:,2] + 4*x[:,3] + 5*x[:,4] -15.5
    return g
def g2(x):
    g = x[:,0]**2 + 2*x[:,1]**2 + x[:,2]**3 + torch.sqrt(x[:,3]+1) + torch.log(x[:,4]+1) -8.6
    return g
def g3(x):
    g = x[:,0]**2*x[:,1]**3 + torch.log(x[:,2]+1) + torch.sqrt(x[:,3]*x[:,4]+1) + torch.exp(x[:,4]/2) -8.2
    return g
def g4(x):
    g = (x[:,0]**2*x[:,1]**3 + torch.log(x[:,2]+1) + torch.sqrt(x[:,3]*x[:,4]+1) + torch.exp(x[:,4]/2))**2/20 -6.0
    return g
#定义非参数部分函数

def inv_func1(beta11,beta12 , g, u, z, x,p):
    '''
    beta: 线性协变量系数
    g:非线性协变量函数
    u:均匀分布随机变量
    z:线性部分协变量
    x:非线性部分协变量
    p:主要事件生成概率
    '''
    hazard = torch.exp(beta11*z[:,0]+beta12*z[:,1]  + g(x))
    y = -torch.log(1-(1-(1-u)**(1/(hazard)))/p)
    return y
#定义主要事件逆概率函数

def inv_func2(beta21,beta22,z,u):
    '''
    beta: 线性协变量系数
    u:均匀分布随机变量
    z:线性部分协变量
    '''
    hazard = torch.exp(beta21*z[:,0]+beta22*z[:,1])
    rate = torch.exp(hazard)
    y = -torch.log(1-u)/rate
    return y
#定义竞争风险事件的逆概率函数
def inv_funcc(a,b,u):
    c=u*(b-a)+a
    return c
#定义删失事件的逆概率函数
def dataproduce(n, beta11, beta12, beta21, beta22, g_index, z_index, seed, p, a, b):
    '''
    n:样本量
    beta:线性协变量系数
    g_index:非线性协变量函数的选择指标
    z_index:线性协变量分布的选择指标
    seed:随机数种子
    mu:删失随机变量参数
    p:生成主要事件概率
    '''
    np.random.seed(seed)
    torch.manual_seed(seed)#设置随机数种子
    sigma = 0.5*torch.ones((5,5)) + 0.5*torch.eye(5)
    mvnorm = st.multivariate_normal(mean=[0,0,0,0,0], cov=sigma)#定义Gaussian copula
    X = torch.from_numpy(2*st.norm.cdf(mvnorm.rvs(n)))
    if z_index==1:
        Z = torch.randint(2,size = [2*n])
    else:
        Z = torch.randn(2*n)/torch.sqrt(torch.tensor([2.0]))+0.5#生成线性协变量
    Z = Z.reshape(n,2)
    if g_index==1:
        g = g1
    if g_index==2:
        g = g2
    if g_index==3:
        g = g3
    if g_index==4:
        g = g4
    #选择函数g
    u =  torch.rand(n)
    t = inv_func1(beta11, beta12, g, u, Z, X,p)#生成事件发生时间
    index2=~(t>0)
    n2=int(sum(index2))
    #确定竞争风险事件的指标集和样本量
    u=torch.rand(n2)
    t2=inv_func2(beta21,beta22,Z[index2],u)
    #生成竞争风险事件的数据
    t[index2]=t2
    epsilon = torch.zeros(n)
    epsilon[index2] = 1
    epsilon = epsilon+1
    #得出事件类型指标
    u = torch.rand(n)
    C = inv_funcc(a,b,u)
    delta = (C >= t)+0.0
    t = torch.minimum(t,C)
    return t.reshape(n,1),epsilon.reshape(n,1),Z.reshape(n,2),X,delta.reshape(n,1)
#定义数据生成函数

def likelihood(t, epsilon, Z, beta, g, delta,G_hat):
    n = len(epsilon)
    t_re = t.reshape((n,1))
    epsilon_re = epsilon.reshape((n,1))
    G = G_hat/G_hat.T
    G[torch.isnan(G)] = 0
    G[torch.isinf(G)] =-0
    R1 = t_re@torch.ones((1,n)).to(device) <= (t_re@torch.ones((1,n)).to(device)).T
    R2 = torch.minimum(t_re@torch.ones((1,n)).to(device) >= (t_re@torch.ones((1,n)).to(device)).T,torch.ones((n,1)).to(device)@(epsilon_re-1).T)
    R2 = torch.minimum(R2,torch.ones((n,1)).to(device)@(delta).T)*G
    R = torch.maximum(R1,R2)
    beta = beta.reshape((len(beta),1))
    g = g.reshape((n,1))
    l = torch.sum(((2-epsilon_re[delta ==1])*((Z@beta)[delta ==1]+(g)[delta ==1]-torch.log((R@torch.exp(Z@beta+g))[delta ==1]))))
    return l
#定义似然函数
def dedata(data):
    T = data[:,0:1]
    epsilon = data[:,1:2]
    Z = data[:,2:4]
    X = data[:,4:9]
    delta = data[:,9:]
    return T,epsilon,Z,X,delta
#将整个数据集分成list
def cv(data,seed):
    train,test = train_test_split(data, train_size=0.80, test_size=0.20,random_state=seed)
    train,valid = train_test_split(train, train_size=0.80, test_size=0.20,random_state=seed)
    return train,valid,test
#划分训练集，验证集，测试集
def G_hat(T,delta):
    kmf = KaplanMeierFitter()
    kmf.fit(T, event_observed=1-delta)
    km = kmf.survival_function_["KM_estimate"]
    return torch.tensor(km.loc[T[:,0]].values.reshape(len(T[:,0]),1)).to(device)
#定义删失变量的估计值

def likelihood_linear(t, epsilon, Z, X, beta, gamma,delta,G_hat):
    n = len(epsilon)
    t_re = t.reshape((n,1))
    epsilon_re = epsilon.reshape((n,1))
    G = G_hat/G_hat.T
    G[torch.isnan(G)] = 0
    G[torch.isinf(G)] =-0
    R1 = t_re@torch.ones((1,n)).to(device) <= (t_re@torch.ones((1,n)).to(device)).T
    R2 = torch.minimum(t_re@torch.ones((1,n)).to(device) >= (t_re@torch.ones((1,n)).to(device)).T,torch.ones((n,1)).to(device)@(epsilon_re-1).T)
    R2 = torch.minimum(R2,torch.ones((n,1)).to(device)@(delta).T)*G
    R = torch.maximum(R1,R2)
    beta = beta.reshape((len(beta),1))
    gamma = gamma.reshape((len(gamma),1))
    l = torch.sum(((2-epsilon_re[delta ==1])*((Z@beta)[delta ==1]+(X@gamma)[delta ==1]-torch.log((R@torch.exp(Z@beta+X@gamma))[delta ==1]))))
    return l
#定义fine and gray model对应的似然函数

class Net_linear(nn.Module):
    def __init__(self):
        super(Net_linear,self).__init__()
        self.beta=nn.Parameter(torch.zeros(2).to(device),requires_grad=True)
        self.gamma = nn.Parameter(torch.zeros(5).to(device),requires_grad=True)
#定义fine and gray model对应的优化网络
        
class Myloss_linear(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, t, epsilon, Z, X, beta, gamma,delta,G_hat):
        return -likelihood_linear(t, epsilon, Z, X, beta,gamma,delta,G_hat)
#定义fine and gray model对应的损失函数

       
def beta0(data,G_hat_train,G_hat_valid):
    train, valid, test = cv(data,1)
    T_train,epsilon_train,Z_train,X_train,delta_train=dedata(train)
    T_valid,epsilon_valid,Z_valid,X_valid,delta_valid=dedata(valid)
    model  = Net_linear().to(device)
    criterion = Myloss_linear()
    optimizer=optim.SGD(model.parameters(),lr=0.01)
    model.train()
    loss1 = (torch.tensor(1e5))
    loss2 = (torch.tensor(1e6))
    for j in range(int(5e2)):
        optimizer.zero_grad()
        loss=criterion(T_train,epsilon_train,Z_train,X_train, model.beta,model.gamma,delta_train,G_hat_train)
        loss2 = criterion(T_valid,epsilon_valid,Z_valid,X_valid, model.beta,model.gamma,delta_valid,G_hat_valid)
        loss1 = torch.min(loss1,loss2)
        if loss2-loss1 > 1e-5:
            break
        if loss1 >0:
            0
        else:
            break
        loss1 = loss2
        loss.backward()
        optimizer.step()
    return model.beta
#定义参数部分初始化函数
class Net1(nn.Module):
    def __init__(self,beta_0):
        super(Net1,self).__init__()
        self.module=nn.Sequential(
            nn.Linear(5,10),nn.ReLU(),
            nn.Linear(10,1))
        self.beta=nn.Parameter(beta_0 ,requires_grad=True)
    def forward(self,x):
        x=self.module(x)
        return x
class Net2(nn.Module):
    def __init__(self,beta_0):
        super(Net2,self).__init__()
        self.module=nn.Sequential(
            nn.Linear(5,10),nn.ReLU(),
            nn.Linear(10,40),nn.ReLU(),
            nn.Linear(40,40),nn.ReLU(),
            nn.Linear(40,20),nn.ReLU(),
            nn.Linear(20,10),nn.ReLU(),
            nn.Linear(10,1))
        self.beta=nn.Parameter(beta_0 ,requires_grad=True)
    def forward(self,x):
        x=self.module(x)
        return x
class Net3(nn.Module):
    def __init__(self,beta_0):
        super(Net3,self).__init__()
        self.module = nn.Sequential(
            nn.Sequential(nn.Linear(5,10), nn.ReLU()),
            nn.Sequential(nn.Linear(10,40), nn.ReLU(), nn.Dropout(0.3)),
            nn.Sequential(nn.Linear(40,40), nn.ReLU(), nn.Dropout(0.3)),
            nn.Sequential(nn.Linear(40,40), nn.ReLU(), nn.Dropout(0.3)),
            nn.Sequential(nn.Linear(40,40), nn.ReLU(), nn.Dropout(0.3)),
            nn.Sequential(nn.Linear(40,20), nn.ReLU(), nn.Dropout(0.3)),
            nn.Sequential(nn.Linear(20,10), nn.ReLU(), nn.Dropout(0.3)),
            nn.Sequential(nn.Linear(10,1))
        )
        self.beta=nn.Parameter(beta_0 ,requires_grad=True)
    def forward(self,x):
        x=self.module(x)
        return x
class Net4(nn.Module):
    def __init__(self,beta_0):
        super(Net4,self).__init__()
        self.module = nn.Sequential(
            nn.Linear(5, 300),
            nn.ReLU(),
            nn.Linear(300, 300),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(300, 300),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(300, 300),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(300, 300),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(300, 300),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(300, 1)
        )
        self.beta=nn.Parameter(beta_0 ,requires_grad=True)
    def forward(self,x):
        x=self.module(x)
        return x
#定义DNN结构

class MyLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, T,epsilon,Z ,beta ,g,delta,G_hat):
        return -likelihood(T,epsilon,Z,beta ,g,delta,G_hat)
#定义损失函数

def chooseNet(index,beta_0):
    if index ==1:
        model=Net1(beta_0 = beta_0)
        lr = 5e-2
    if index ==2:
        model=Net2(beta_0 = beta_0)
        lr = 1e-3
    if index ==3:
        model=Net3(beta_0 = beta_0)
        lr = 1e-3
    if index ==4:
        model=Net4(beta_0 = beta_0)
        lr = 5e-4
    return model.to(device),lr
#定义网络选择函数
def REg(g_0, g_hat):
    g_hat=g_hat.reshape([len(g_0),])
    y = torch.sqrt(torch.mean(((g_hat-torch.mean(g_hat))-g_0)**2)/(torch.mean((g_0)**2)))
    return y
class h_net(nn.Module):
    def __init__(self):
        super(h_net, self).__init__()
        self.module=nn.Sequential(
            nn.Linear(1,5),nn.ReLU(),
            nn.Linear(5,10),nn.ReLU(),
            nn.Linear(10,10),nn.ReLU(),
            nn.Linear(10,5),nn.ReLU(),
            nn.Linear(5,1),)
    def forward(self,x):
        x=self.module(x)
        return x 

class g_net(nn.Module):
    def __init__(self):
        super(g_net, self).__init__()
        self.module=nn.Sequential(
            nn.Linear(5,10),nn.ReLU(),
            nn.Linear(10,40),nn.ReLU(),
            nn.Linear(40,40),nn.ReLU(),
            nn.Linear(40,20),nn.ReLU(),
            nn.Linear(20,10),nn.ReLU(),
            nn.Linear(10,1))
    def forward(self,x):
        x=self.module(x)
        return x
def VE_loss_func(epsilon,delta,Z,h,g):
    l = torch.mean((2-epsilon)*delta*(Z-h-g)**2)
    return l
class VE_loss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, epsilon,delta,Z,h,g):
        return VE_loss_func(epsilon,delta,Z,h,g)
    
def estimation(Net_index, train, valid, test,G_hat_train,G_hat_valid,G_hat_train0,G_hat_valid0,g):
    window = []
    positive = 0
    T_train,epsilon_train,Z_train,X_train,delta_train=dedata(train)
    T_test,epsilon_test,Z_test,X_test,delta_test=dedata(test)
    T_valid,epsilon_valid,Z_valid,X_valid,delta_valid=dedata(valid)
    beta_0 = beta0(train,G_hat_train0,G_hat_valid0)
    '''
    model = Net(beta_0=beta_0)
    criterion = MyLoss()
    lr = 1e-3
    optimizer = torch.optim.Adam([{'params':model.module.parameters(),'lr':lr},
                                {'params':model.beta,'lr':50*lr}])
    '''
    model,lr = chooseNet(Net_index,beta_0)
    criterion = MyLoss()
    optimizer = torch.optim.Adam([{'params':model.module.parameters(),'lr':lr},
                                {'params':model.beta,'lr':50*lr}])
    loss1 = (torch.tensor(1e5))
    loss2 = (torch.tensor(1e6))
    loss3 = (torch.tensor(1e5))
    loss4 = (torch.tensor(1e6))
    for j in range(int(1000)):
        optimizer.zero_grad()
        g_train = model(X_train)
        loss=criterion(T_train,epsilon_train,Z_train,model.beta ,g_train,delta_train,G_hat_train)
        loss1 = loss
        if torch.isnan(loss2):
            break
        g_valid = model(X_valid)
        loss2 = criterion(T_valid,epsilon_valid,Z_valid,model.beta ,g_valid,delta_valid,G_hat_valid)
        g_test = model(X_test)
        loss3 = criterion(T_valid,epsilon_valid,Z_valid,model.beta ,g_valid,delta_valid,G_hat_valid)
        loss4 = torch.min(loss3,loss4)
        window.append(loss2)
        d = 10
        if j >100:
            criterion_set = torch.mean(torch.tensor(window)[j-d:j]-torch.tensor(window)[j-d-1:j-1])
            if criterion_set>0:
                positive = positive+1
                if loss3 - loss4 > 0.015*torch.abs(loss4):
                    break
                if positive > 50:
                    break
        loss.backward()
        optimizer.step()
    return loss2, model.beta[0], model.beta[1], REg(g(X_test),g_test),g_test,model


def variance_estimation(train, valid, test):
    criterion_ve = VE_loss()
    T_train,epsilon_train,Z_train,X_train,delta_train=dedata(train)
    T_test,epsilon_test,Z_test,X_test,delta_test=dedata(test)
    T_valid,epsilon_valid,Z_valid,X_valid,delta_valid=dedata(valid)
    h_net_1=h_net().to(device)
    h_net_2=h_net().to(device)
    g_net_1=g_net().to(device)
    g_net_2=g_net().to(device)
    optimizer_1 = torch.optim.Adam(list(h_net_1.parameters())+list(g_net_1.parameters()), lr=0.001)
    h_net_1.train()
    g_net_1.train()
    loss1 = (torch.tensor(1e5))
    loss2 = (torch.tensor(1e6))
    for j in range(int(1e4)):
        optimizer_1.zero_grad()
        loss = criterion_ve(epsilon_train, delta_train, Z_train[:,[0]], h_net_1(T_train), g_net_1(X_train))
        loss2 = criterion_ve(epsilon_valid, delta_valid, Z_valid[:,[0]], h_net_1(T_valid), g_net_1(X_valid))
        loss1 = torch.min(loss1,loss2)
        if loss2-loss1 > 1e-2:
            break
        if loss1 >0:
            0
        else:
            break
        loss1 = loss2
        loss.backward()
        optimizer_1.step()
    optimizer_2 = torch.optim.Adam(list(h_net_2.parameters())+list(g_net_2.parameters()), lr=0.001)
    h_net_2.train()
    g_net_2.train()
    loss1 = (torch.tensor(1e5))
    loss2 = (torch.tensor(1e6))
    for j in range(int(1e4)):
        optimizer_2.zero_grad()
        loss = criterion_ve(epsilon_train, delta_train, Z_train[:,[1]], h_net_2(T_train), g_net_2(X_train))
        loss2 = criterion_ve(epsilon_valid, delta_valid, Z_valid[:,[1]], h_net_2(T_valid), g_net_2(X_valid))
        loss1 = torch.min(loss1,loss2)
        if loss2-loss1 > 2e-2:
            break
        if loss1 >0:
            0
        else:
            break
        loss1 = loss2
        loss.backward()
        optimizer_2.step()
    a = torch.mean((2-epsilon_train)*delta_train*(Z_train[:,[0]]-h_net_1(T_train)-g_net_1(X_train))**2)
    b = torch.mean((2-epsilon_train)*delta_train*(Z_train[:,[0]]-h_net_1(T_train)-g_net_1(X_train))*(Z_train[:,[1]]-h_net_2(T_train)-g_net_2(X_train)))
    c = b 
    d = torch.mean((2-epsilon_train)*delta_train*(Z_train[:,[1]]-h_net_2(T_train)-g_net_2(X_train))**2)
    std1 = torch.sqrt(d/(a*d-b*c))
    std2 = torch.sqrt(a/(a*d-b*c))
    return std1,std2
def CI_cover(n,estimation, std, parameter):
    n = torch.tensor(n)
    cover = 0
    if (estimation-1.96*std/torch.sqrt(n)<= parameter)&(estimation+1.96*std/torch.sqrt(n)>= parameter):
        cover = cover+1
    return cover
def riskmatrix(epsilon,T,delta,G_hat):
    n = len(epsilon)
    t_re = T.reshape((n,1))
    epsilon_re = epsilon.reshape((n,1))
    G = G_hat/G_hat.T
    G[torch.isnan(G)] = 0
    G[torch.isinf(G)] =-0
    R1 = t_re@torch.ones((1,n)).to(device) <= (t_re@torch.ones((1,n)).to(device)).T
    R2 = torch.minimum(t_re@torch.ones((1,n)).to(device) >= (t_re@torch.ones((1,n)).to(device)).T,torch.ones((n,1)).to(device)@(epsilon_re-1).T)
    R2 = torch.minimum(R2,torch.ones((n,1)).to(device)@(delta).T)*G
    R = torch.maximum(R1,R2)
    return R
def riskmatrix_add(epsilon,T,delta,G_hat):
    n = len(epsilon)
    t_re = T.reshape((n,1))
    epsilon_re = epsilon.reshape((n,1))
    G = G_hat/G_hat.T
    G[torch.isnan(G)] = 0
    G[torch.isinf(G)] =-0
    R1 = t_re@torch.ones((1,n)).to(device) < (t_re@torch.ones((1,n)).to(device)).T
    R2 = torch.minimum(t_re@torch.ones((1,n)).to(device) >= (t_re@torch.ones((1,n)).to(device)).T,torch.ones((n,1)).to(device)@(epsilon_re-1).T)
    R2 = torch.minimum(R2,torch.ones((n,1)).to(device)@(delta).T)*G
    R = torch.maximum(R1,R2)
    return R
def TP(R,M):
    n = len(M)
    expM_vector = torch.exp(M).reshape(1,n)
    Y_expM_matrix = R*expM_vector#列时间变化，行样本变化
    M_order_matrix = (M.reshape(n,1)>M.reshape(1,n)).double()
    Numerator = Y_expM_matrix@M_order_matrix#T,M
    Denominator = Y_expM_matrix@torch.ones((n,n)).to(device)
    TP  = Numerator/Denominator
    return TP
def FP(R_add,M):
    n = len(M)
    M_order_matrix = (M.reshape(n,1)>M.reshape(1,n)).double()
    Numerator = R_add@M_order_matrix#T,M
    Denominator = R_add@torch.ones((n,n)).to(device)
    Denominator[Denominator==0]=1
    FP = Numerator/Denominator
    return FP
def auc_t(M,TPR,FPR):
    n = len(M)
    auc_array = torch.zeros(n)
    for i in range(n):
        sorted_index = torch.argsort(FPR[i])
        fpr_list_sorted =  torch.tensor(FPR[i])[sorted_index]
        tpr_list_sorted = torch.tensor(TPR[i])[sorted_index]
        auc_array[i] = integrate.trapz(y=tpr_list_sorted, x=fpr_list_sorted)
    return auc_array
def Stof(S,t):
    index = torch.argsort(t[:,0])
    t_ordered = t[index]
    S_ordered = S[index]
    f = (torch.concatenate((torch.ones((1,len(S_ordered))).to(device), S_ordered[:-1])) -S_ordered )/(t_ordered- torch.concatenate((torch.tensor([[0]]).to(device), t_ordered[:-1])))
    return f
def Gamma_hat(t, epsilon, Z,beta, g, delta,G_hat):
    n = len(epsilon)
    t_re = t.reshape((n,1))
    epsilon_re = epsilon.reshape((n,1))
    G = G_hat/G_hat.T
    G[torch.isnan(G)] = 0
    G[torch.isinf(G)] =-0
    R1 = t_re@torch.ones((1,n)).to(device) <= (t_re@torch.ones((1,n)).to(device)).T
    R2 = torch.minimum(t_re@torch.ones((1,n)).to(device) >= (t_re@torch.ones((1,n)).to(device)).T,torch.ones((n,1)).to(device)@(epsilon_re-1).T)
    R2 = torch.minimum(R2,torch.ones((n,1).to(device)@(delta).T))*G
    R = torch.maximum(R1,R2)
    beta = beta.reshape((len(beta),1))
    g = g.reshape((n,1))
    S=(R@torch.exp(Z@beta+g))/n
    Gamma = ((t>=(t.T)).double()@((2-epsilon)*delta/S))/n
    return Gamma
def f_hat(t, epsilon, Z,beta, g, delta):
    Gamma = Gamma_hat(t, epsilon, Z,beta, g, delta)
    n = len(epsilon)
    beta = beta.reshape((len(beta),1))
    g = g.reshape((n,1))
    S = torch.exp(-(Gamma)@torch.exp(Z@beta+g).T)
    f = Stof(S,t)
    return torch.mean(f,axis = 1)
def S_hat(t, epsilon, Z,beta, g, delta):
    Gamma = Gamma_hat(t, epsilon, Z,beta, g, delta)
    n = len(epsilon)
    beta = beta.reshape((len(beta),1))
    g = g.reshape((n,1))
    S = torch.exp(-(Gamma)@torch.exp(Z@beta+g).T)
    index = torch.argsort(t[:,0])
    S_ordered = S[index]
    return torch.mean(S_ordered,axis = 1)
def Gamma_real(t,p):
    Gamma = -torch.log(1-p*(1-torch.exp(-t)))
    return Gamma
def f_real(t,  Z,beta, g, p):
    n = len(t)
    Gamma = Gamma_real(t, p)
    beta = beta.reshape((len(beta),1))
    g = g.reshape((len(g),1))
    S = torch.exp(-(Gamma)@torch.exp(Z@beta+g).T)
    f = Stof(S,t)
    return torch.mean(f,axis = 1)
def S_real(t,  Z,beta, g, p):
    n = len(t)
    Gamma = Gamma_real(t, p)
    beta = beta.reshape((len(beta),1))
    g = g.reshape((len(g),1))
    S = torch.exp(-(Gamma)@torch.exp(Z@beta+g).T)
    index = torch.argsort(t[:,0])
    S_ordered = S[index]
    return torch.mean(S_ordered,axis = 1)

def C_tau(auc_array,U,U_ordered, X, Z, g_, p):  
    beta = torch.Tensor([1.0,1.0]).to(device)
    g = g_(X)
    f = f_real(U.reshape((len(U),1)),  Z,beta, g, p)
    S = S_real(U.reshape((len(U),1)),  Z,beta, g, p)
    t_index = torch.sort(U).indices
    t_ordered = U[t_index]
    sub_index = torch.where(torch.isin(t_ordered, U_ordered))
    w = (2*f*S)[sub_index]
    W = torch.trapz(y=w, x=t_ordered[sub_index])
    w_tau = w/W
    C = torch.trapz(y=auc_array*w_tau, x=t_ordered[sub_index])
    return C
def cubic_spline(q,u,m):#q+4:样条基个数，u输入向量
    T = np.around(np.linspace(0,0.8,q),2)
    n = len(u)
    B = np.zeros((n,q))
    for i in range(n):
        B[i,0] = 1
        B[i,1] = u[i]
        B[i,2] = u[i]**2
        B[i,3] = u[i]**3
        for j in range(q):
            if u[i] > T[j]:
                B[i,j] = (u[i] - T[j])**3
    return B
def outcome_to_df(outcomes,i):
    outcomes_colname = ['beta1', 'beta2', 'REg', 'std1', 'std2', 'cover_rate1', 'cover_rate2', 'AUC(25%)','AUC(50%)','AUC(75%)','C^tau']
    df_outcomes_DP = (pd.DataFrame(np.array(outcomes)[:,i:])).rename(columns=dict(zip((pd.DataFrame(np.array(outcomes)[:,i:])), outcomes_colname)))
    return df_outcomes_DP
def TPR_FPR(U_test,M,M0):
    U_ordered = U_test[(U_test>0)&(U_test<U_test.max())].sort().values
    Y_t = (~(U_test.reshape((len(U_test),1)) < U_ordered.reshape((1,len(U_ordered))))).int().to(device)
    '''
    Y_i(t_j)对应的矩阵(i,j)
    '''
    Yt_eM = Y_t*(torch.exp(M0).reshape((len(M0),1)))
    W_t = Yt_eM.sum(axis =0).reshape((1,len(U_ordered)))
    TPR = ((M.reshape(len(M),1)>M.sort().values.reshape(1,len(M))).double().T)@(Yt_eM/W_t)
    '''
    mi,tj对应(i,j)
    '''
    Y_t_add = (~(U_test.reshape((len(U_test),1)) <= U_ordered.reshape((1,len(U_ordered))))).int().to(device)
    W_t_add = Y_t_add.sum(axis =0).reshape((1,len(U_ordered)))
    FPR = ((M.reshape(len(M),1)>M.sort().values.reshape(1,len(M))).double().T)@(Y_t_add/W_t_add)
    return torch.flip(TPR,[0]),torch.flip(FPR,[0])
def Uproduce(n, beta11, beta12, beta21, beta22, g_index, z_index, seed, p, a, b):
    '''
    n:样本量
    beta:线性协变量系数
    g_index:非线性协变量函数的选择指标
    z_index:线性协变量分布的选择指标
    seed:随机数种子
    mu:删失随机变量参数
    p:生成主要事件概率
    '''
    np.random.seed(seed)
    torch.manual_seed(seed)#设置随机数种子
    sigma = 0.5*torch.ones((5,5)) + 0.5*torch.eye(5)
    mvnorm = st.multivariate_normal(mean=[0,0,0,0,0], cov=sigma)#定义Gaussian copula
    X = torch.from_numpy(2*st.norm.cdf(mvnorm.rvs(n)))
    if z_index==1:
        Z = torch.randint(2,size = [2*n])
    else:
        Z = torch.randn(2*n)/torch.sqrt(torch.tensor([2.0]))+0.5#生成线性协变量
    Z = Z.reshape(n,2)
    if g_index==1:
        g = g1
    if g_index==2:
        g = g2
    if g_index==3:
        g = g3
    if g_index==4:
        g = g4
    #选择函数g
    u =  torch.rand(n)
    t = inv_func1(beta11, beta12, g, u, Z, X,p)#生成事件发生时间
    return t

In [6]:
samplesize_set = [1000,2000]
g_index_set  =  [1,2,3,4]
g__set = [g1,g2,g3,g4]
z_index_set =[1,2]
b_set = [0.15,0.5]
#b=0.15对应60%删失率，b=0.5对应40%删失率
p_set = [0.3,0.7]
folder_path = r'D:\simulations'
for samplesize in samplesize_set:
    for g_index in g_index_set:
        g_ = g__set[g_index-1]
        for z_index in z_index_set:
            for b in b_set:
                for p in p_set:
                    excelname = ' '+str(samplesize)+' '+str(g_index)+' '+str(z_index)+' '+str(b)+' '+str(p)+'.xlsx'
                    outcomes_DP = []
                    outcomes_spline = []
                    outcomes_Linear = []
                    cover_num_DP1 = 0
                    cover_num_DP2 = 0
                    cover_num_Linear1 = 0
                    cover_num_Linear2 = 0
                    cover_num_spline1 = 0
                    cover_num_spline2 = 0
                    for i_  in range(100): 
                        print(i_)
                        loss_kk = []
                        beta1_kk = []
                        beta2_kk = []
                        REg_kk = []
                        g_kk =[]
                        model_kk = []
                        data = torch.hstack(dataproduce(n=samplesize, beta11=1.0, beta12=1.0, beta21=-0.5, beta22=0.5, g_index=g_index, z_index=z_index, seed=i_, p=p,a=0,b=b))
                        data_np = data
                        train, valid, test = cv(data,1)
                        train_np = train.detach().numpy()
                        valid_np = valid.detach().numpy()
                        test_np = test.detach().numpy()
                        train0, valid0, test0 = cv(train,1)
                        T_train0,epsilon_train0,Z_train,X_train0,delta_train0=dedata(train0)
                        T_valid0,epsilon_valid0,Z_valid0,X_valid0,delta_valid0=dedata(valid0)
                        T_train,epsilon_train,Z_train,X_train,delta_train=dedata(train)
                        T_test, epsilon_test, Z_test, X_test, delta_test = dedata(test)
                        T_valid,epsilon_valid,Z_valid,X_valid,delta_valid=dedata(valid)
                        T, epsilon, Z, X, delta = dedata(data)
                        G_hat_train = G_hat(T_train,delta_train)
                        G_hat_train0 = G_hat(T_train0,delta_train0)
                        G_hat_valid = G_hat(T_valid,delta_valid)
                        G_hat_valid0 = G_hat(T_valid0,delta_valid0)
                        G_hat_test = G_hat(T_test,delta_test)
                        G_hat_data = G_hat(T,delta)
                        data = data.to(device)
                        train, valid, test = cv(data,1)
                        T, epsilon, Z, X, delta = dedata(data)
                        test_in_data_index = []
                        for i in range(len(test[:,0])):
                            test_in_data_index.append(torch.where(torch.isin(data[:,0],test[i,0]))[0][0])
                        torch.Tensor(test_in_data_index).int()
                        '''
                        找出test在整个data里对应的index
                        '''
                        U_test =Uproduce(n=samplesize, beta11=1.0, beta12=1.0, beta21=-0.5, beta22=0.5, g_index=g_index, z_index=z_index, seed=0, p=p,a=0,b=b)[test_in_data_index].to(device)
                        U_test[torch.isnan(U_test)]=1e4
                        U = Uproduce(n=samplesize, beta11=1.0, beta12=1.0, beta21=-0.5, beta22=0.5, g_index=g_index, z_index=z_index, seed=0, p=p,a=0,b=b).to(device)
                        U[torch.isnan(U)]=1e4
                        U_unordered = U_test[(U_test>0)&(U_test<U_test.max())]
                        U_ordered = U_test[(U_test>0)&(U_test<U_test.max())].sort().values
                        len_T = len(U_ordered)
                        M0 = (Z_test[:,0]+Z_test[:,1]+g_(X_test)).to(device)
                        for N_index in range(4):
                            for kk in range(5):
                                loss, beta1,beta2,REg_,g ,model = estimation(N_index+1,train, valid, test,G_hat_train,G_hat_valid,G_hat_train0,G_hat_valid0,g_)
                                if torch.isnan(loss):
                                    break
                                loss_kk.append(loss)
                                beta1_kk.append(beta1)
                                beta2_kk.append(beta2)
                                REg_kk.append(REg_)
                                g_kk.append(g)
                                model_kk.append(model)
                        index_kk = torch.where(torch.min(torch.tensor(loss_kk))==torch.tensor(loss_kk))[0]
                        std1,std2=variance_estimation(train, valid, test)
                        if CI_cover(samplesize,beta1_kk[index_kk].item(),std1,1.0) == 1:
                            cover_num_DP1 = cover_num_DP1+1
                        if CI_cover(samplesize,beta2_kk[index_kk].item(),std2,1.0) == 1:
                            cover_num_DP2 = cover_num_DP2+1
                        T_test, epsilon_test, Z_test, X_test, delta_test = dedata(test)
                        M = beta1_kk[index_kk]*Z_test[:,0]+beta2_kk[index_kk]*Z_test[:,1]+g_kk[index_kk].reshape(int(samplesize*0.2))-g_kk[index_kk].reshape(int(samplesize*0.2)).mean()
                        TPR,FPR = TPR_FPR(U_test,M,M0)
                        auc_array = torch.zeros(len_T).to(device)
                        for i in range(len_T):
                            auc_array[i]=torch.trapz(y=TPR[:,i], x=FPR[:,i])
                        num  = len(auc_array)
                        q1 = auc_array[int(num/4)].item()
                        q2 = auc_array[int(2*num/4)].item()
                        q3 = auc_array[int(3*num/4)].item()
                        Ctau=C_tau(auc_array,U,U_ordered, X, Z, g_, p).item()
                        outcome_DP = [loss_kk[index_kk].item(),beta1_kk[index_kk].item(),beta2_kk[index_kk].item(),REg_kk[index_kk].item(),(std1/np.sqrt(samplesize)).item(),(std2/np.sqrt(samplesize)).item(),cover_num_DP1,cover_num_DP2,q1,q2,q3,Ctau]
                        outcomes_DP.append(outcome_DP)
                        print('DP',beta1_kk[index_kk].item(),beta2_kk[index_kk].item(),REg_kk[index_kk].item(),(1.96*std1/np.sqrt(samplesize)).item(),(1.96*std2/np.sqrt(samplesize)).item(),cover_num_DP1,cover_num_DP2,q1,q2,q3,Ctau)
                        T_train,epsilon_train,Z_train,X_train,delta_train=dedata(train_np)
                        T_test,epsilon_test,Z_test,X_test,delta_test=dedata(test_np)
                        T_valid,epsilon_valid,Z_valid,X_valid,delta_valid=dedata(valid_np)
                        df = pd.DataFrame(np.hstack([T_train,epsilon_train*delta_train,Z_train,X_train]))
                        # 创建一个新的列标列表
                        new_columns = ['ftime', 'fstatus', 'Z1', 'Z2', 'X1', 'X2', 'X3', 'X4', 'X5']

                        # 使用rename方法将列标更改为新列表中的值
                        df = df.rename(columns=dict(zip(df.columns, new_columns)))
                        a = cmprsk.crr(failure_time = df['ftime'],failure_status = df['fstatus'],static_covariates =df[['Z1','Z2','X1','X2','X3','X4','X5']], failcode=1, cencode=0)
                        g_hat = torch.tensor(X_test@a.summary.coefficients.values[2:])
                        g_0 = g_(torch.tensor(X_test))
                        std1 = a.summary["std"][0]
                        std2 = a.summary["std"][1]
                        if CI_cover(samplesize,a.summary.coefficients.values[0],std1*np.sqrt(samplesize),1.0) == 1:
                            cover_num_Linear1 = cover_num_Linear1+1
                        if CI_cover(samplesize,a.summary.coefficients.values[1],std2*np.sqrt(samplesize),1.0) == 1:
                            cover_num_Linear2 = cover_num_Linear2+1
                        M = torch.tensor(Z_test@a.summary.coefficients.values[0:2]+X_test@a.summary.coefficients.values[2:]-(X_test@a.summary.coefficients.values[2:]).mean()).to(device)
                        TPR,FPR = TPR_FPR(U_test,M,M0)
                        auc_array = torch.zeros(len_T).to(device)
                        for i in range(len_T):
                            auc_array[i]=torch.trapz(y=TPR[:,i], x=FPR[:,i])
                        num  = len(auc_array)
                        q1 = auc_array[int(num/4)].item()
                        q2 = auc_array[int(2*num/4)].item()
                        q3 = auc_array[int(3*num/4)].item()
                        Ctau=C_tau(auc_array,U,U_ordered, X, Z, g_, p).item()
                        outcome_Linear = [a.summary.coefficients.values[0],a.summary.coefficients.values[1],REg(g_0,g_hat).item(),std1,std2,cover_num_Linear1,cover_num_Linear2,q1,q2,q3,Ctau]
                        outcomes_Linear.append(outcome_Linear)
                        print('Linear',a.summary.coefficients.values[0],a.summary.coefficients.values[1],REg(g_0,g_hat).item(),1.96*std1,1.96*std2,cover_num_Linear1,cover_num_Linear2,q1,q2,q3,Ctau)
                        q = 5
                        m = 3
                        spline_df = np.hstack([cubic_spline(q,X_train[:,0],m),cubic_spline(q,X_train[:,1],m),cubic_spline(q,X_train[:,2],m),cubic_spline(q,X_train[:,3],m),cubic_spline(q,X_train[:,4],m)])
                        spline_df = np.hstack([Z_train,spline_df])
                        spline_df = pd.DataFrame(spline_df)
                        df = pd.DataFrame(np.hstack([T_train,epsilon_train*delta_train,Z_train,X_train]))
                        # 创建一个新的列标列表
                        new_columns = ['ftime', 'fstatus', 'Z1', 'Z2', 'X1', 'X2', 'X3', 'X4', 'X5']
                        # 使用rename方法将列标更改为新列表中的值
                        df = df.rename(columns=dict(zip(df.columns, new_columns)))
                        a = cmprsk.crr(failure_time = df['ftime'],failure_status = df['fstatus'],static_covariates =spline_df, failcode=1, cencode=0)
                        X_test_spline = np.hstack([cubic_spline(q,X_test[:,0],m),cubic_spline(q,X_test[:,1],m),cubic_spline(q,X_test[:,2],m),cubic_spline(q,X_test[:,3],m),cubic_spline(q,X_test[:,4],m)])
                        g_hat = torch.tensor(X_test_spline@a.summary.coefficients.values[2:])
                        g_0 = g_(torch.tensor(X_test))
                        std1 = a.summary["std"][0]
                        std2 = a.summary["std"][1]
                        if CI_cover(samplesize,a.summary.coefficients.values[0],std1*np.sqrt(samplesize),1.0) == 1:
                            cover_num_spline1 = cover_num_spline1+1
                        if CI_cover(samplesize,a.summary.coefficients.values[1],std2*np.sqrt(samplesize),1.0) == 1:
                            cover_num_spline2 = cover_num_spline2+1
                        M = torch.tensor(Z_test@a.summary.coefficients.values[0:2]+X_test_spline@a.summary.coefficients.values[2:]-(X_test_spline@a.summary.coefficients.values[2:]).mean()).to(device)
                        TPR,FPR = TPR_FPR(U_test,M,M0)
                        auc_array = torch.zeros(len_T).to(device)
                        for i in range(len_T):
                            auc_array[i]=torch.trapz(y=TPR[:,i], x=FPR[:,i])
                        num  = len(auc_array)
                        q1 = auc_array[int(num/4)].item()
                        q2 = auc_array[int(2*num/4)].item()
                        q3 = auc_array[int(3*num/4)].item()
                        Ctau=C_tau(auc_array,U,U_ordered, X, Z, g_, p).item()
                        outcome_spline= [a.summary.coefficients.values[0],a.summary.coefficients.values[1],REg(g_0,g_hat).item(),std1,std2,cover_num_spline1,cover_num_spline2,q1,q2,q3,Ctau]
                        outcomes_spline.append(outcome_spline)
                        print('spline',a.summary.coefficients.values[0],a.summary.coefficients.values[1],REg(g_0,g_hat).item(),1.96*std1,1.96*std2,cover_num_spline1,cover_num_spline2,q1,q2,q3,Ctau)
                    df_outcomes_DP = outcome_to_df(outcomes_DP,1)
                    df_outcomes_Linear = outcome_to_df(outcomes_Linear,0)
                    df_outcomes_spline = outcome_to_df(outcomes_spline,0)
                    df_outcomes_DP['cover_rate1']=df_outcomes_DP['cover_rate1'][99]
                    df_outcomes_Linear['cover_rate1']=df_outcomes_Linear['cover_rate1'][99]
                    df_outcomes_spline['cover_rate1']=df_outcomes_spline['cover_rate1'][99]
                    df_outcomes_DP['cover_rate2']=df_outcomes_DP['cover_rate2'][99]
                    df_outcomes_Linear['cover_rate2']=df_outcomes_Linear['cover_rate2'][99]
                    df_outcomes_spline['cover_rate2']=df_outcomes_spline['cover_rate2'][99]
                    df_mean = pd.DataFrame([df_outcomes_DP.mean(),df_outcomes_Linear.mean(),df_outcomes_spline.mean()])
                    df_std = pd.DataFrame([df_outcomes_DP.std(),df_outcomes_Linear.std(),df_outcomes_spline.std()])
                    filepath = os.path.join(folder_path,excelname)
                    writer = pd.ExcelWriter(filepath)
                    # 将每个 DataFrame 写入不同的工作表（sheet）中
                    df_outcomes_DP.to_excel(writer, sheet_name='DP')
                    df_outcomes_Linear.to_excel(writer, sheet_name='Linear')
                    df_outcomes_spline.to_excel(writer, sheet_name='spline')
                    df_mean.to_excel(writer, sheet_name='mean')
                    df_std.to_excel(writer, sheet_name='std')
                    writer.save()      

0
DP 0.9600492459483265 1.1546755132020308 0.09318914396982556 0.8834723390965932 0.46001276978521627 1 1 0.9542306838082555 0.9554703591490092 0.9402954030860531 0.9484899001687999
Linear 0.9601050314967353 1.1547799190182866 0.0932463296557707 0.2773823777109898 0.298843314441489 1 1 0.9542306836664338 0.9554703559479912 0.940295313088309 0.9484896574926766
spline 1.0170259785953137 1.2265884568439402 0.1775185095291591 0.3035121433614764 0.31819273073151944 1 1 0.9522455929602589 0.954620495897248 0.9406155497174213 0.9465654631476077
1
DP 1.027040756816662 0.9501453334162604 0.04560066542464417 0.3382056817698626 2.5152987474711925 2 2 0.9724133527376935 0.9728507436255385 0.9700687439564015 0.9708919894213365
Linear 1.0270541816543242 0.9501457994561113 0.04559376357802273 0.3220838984343338 0.2951311301032851 2 2 0.9724133527376935 0.9728507436255385 0.9700687439564015 0.9708919894213365
spline 1.103842768817601 0.966807199144466 0.17850294227471292 0.34378694062075277 0.31848978

C:\Users\znj\AppData\Local\Temp\ipykernel_17388\1473708906.py:184: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


0
DP 1.0169135499994868 1.2460435362253288 0.09547852842965758 0.305609186943528 0.37697633376211104 1 1 0.9522652455927721 0.9631388522579951 0.9522894917006346 0.9471699783953547
Linear 1.0169165016028152 1.246042447121479 0.09551202285964819 0.2586087404663728 0.287162427222481 1 1 0.9522652455927721 0.9631388522579951 0.9522894917006346 0.9471699783953547
spline 1.0823199226777236 1.3211843410288646 0.15295908733881144 0.2833176159234714 0.3015858057784846 1 0 0.9507986707310941 0.9623759814049466 0.9509328195650645 0.9455816748851003
1
DP 0.941802867394373 0.9049281503682733 0.09110537487674562 0.32868618116690335 0.4106736527384698 2 2 0.9721708005161265 0.971954260744738 0.9691295516236171 0.9702706359412197
Linear 0.9363271143320815 0.9246503565934308 0.03193447638031376 0.28975079387380664 0.2561448607799347 2 2 0.9722641741376793 0.9721065802404543 0.9693724121076878 0.9704495605570108
spline 1.0382194048264115 0.9947272139379927 0.15920730867587396 0.3026859049737636 0.28261

RuntimeError: CUDA error: unknown error
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.